In [1]:
import cv2
import sys
from random import randint

## Create a method to use to create the tracker

In [2]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
 
def createTrackerByName(trackerType):
  # Create a tracker based on tracker name
  if trackerType == trackerTypes[0]:
    tracker = cv2.TrackerBoosting_create()
  elif trackerType == trackerTypes[1]: 
    tracker = cv2.TrackerMIL_create()
  elif trackerType == trackerTypes[2]:
    tracker = cv2.TrackerKCF_create()
  elif trackerType == trackerTypes[3]:
    tracker = cv2.TrackerTLD_create()
  elif trackerType == trackerTypes[4]:
    tracker = cv2.TrackerMedianFlow_create()
  elif trackerType == trackerTypes[5]:
    tracker = cv2.TrackerGOTURN_create()
  elif trackerType == trackerTypes[6]:
    tracker = cv2.TrackerMOSSE_create()
  elif trackerType == trackerTypes[7]:
    tracker = cv2.TrackerCSRT_create()
  else:
    tracker = None
    print('Incorrect tracker name')
    print('Available trackers are:')
    for t in trackerTypes:
      print(t)
     
  return tracker

 ## Attempt to read the video

In [3]:
#Set video0 to a video with ~4 minutes of content.
#Set video1 to a car chase
video0 = "ball rebounding and bouncing all around the screen with trails and music 1080p HD Animation.mp4"
video1 = "1-Stolen Dodge Hellcat Outruns Chopper in Houston Police Chase! Driver Almost Makes it.mp4"
video = video1
capture = cv2.VideoCapture(video)

#Attempt to read a frame
success, frame = capture.read()
if not success:
    print("Video doesn't exist.")
    sys.exit(1)

## Draw bounding boxes around the object to be tracked

In [4]:
boundBoxes = []
# color is going to be initialized to a random color for every object
colors = []

# A loop to run the object selection till "q" is pressed
while not True:
    boundbox = cv2.selectROI('tracker', frame)
    boundBoxes.append(boundbox)
    colors.append((randint(0,255), randint(0, 255), randint(0, 255)))
    print("Press q/ESC to quit selecting boxes and start tracking")
    print("Press Space to set the next object to select")
    k = cv2.waitKey(0)
    if (k == 113 or k==27):
        break        
        
cv2.destroyWindow("tracker")
print("Selected bounding boxes{}".format(boundBoxes))

Selected bounding boxes[]


## Set bounding box and color to a fixed location so test is reproducible

In [5]:
if video==video0:
    boundBoxes = [(0, 0, 11, 16)]
    colors = [(44, 156, 222)]
if video ==video1:
    boundBoxes = [(258, 156, 76, 32)]
    colors = [(44, 156, 222)]

## Initialize a Multi-tracker to track the boundingBoxes

In [6]:
['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
trackerType = "TLD"

#Create the multitracker object
multiTracker = cv2.MultiTracker_create()

#initialize the multitracker
for boundbox in boundBoxes:
    multiTracker.add(createTrackerByName(trackerType), frame, boundbox)

## Process the video and begin object tracking

In [7]:
def run_tracking(capture, multiTracker, trackerType):
    frameCount = 1
    #Process video and track objects
    while capture.isOpened():
        frameCount+=1
        success, frame = capture.read()
        if not success:
            break
        #get updated location of objects being tracked in subsequent fram
        success, boxes = multiTracker.update(frame)
        
        #Draw box around tracked objects
        for i, newbox in enumerate(boxes):
            p1 = (int(newbox[0]), int(newbox[1]))
            p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
            cv2.rectangle(frame, p1, p2, colors[i], 2, 1)

        #Show the fram
        cv2.imshow(trackerType, frame)

        #Quit if the Escape button is pressed
        if cv2.waitKey(1) & 0xFF ==27:
            break
    cv2.destroyAllWindows()
    print(frameCount)

In [8]:
%time run_tracking(capture, multiTracker, trackerType)

2241
Wall time: 2min 56s


In [9]:
print(success)
cv2.destroyAllWindows()

True
